In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Simulation parameters
f = 10000  # Frequency (Hz)
E_rms = np.linspace(500, 5000, 10) / (np.sqrt(2) * np.pi * 0.1)  # Root mean square electric field (V/m)
L = 1  # Length of discharge (m)
d = 0.1  # Distance between electrodes (m)
W = 0.1  # Width of electrodes (m)
A = L * W  # Surface area of electrodes (m^2)
mu_0 = 4 * np.pi * 10**(-7)  # Permeability of free space (H/m)
epsilon_0 = 8.854 * 10**(-12)  # Permittivity of free space (F/m)
epsilon_r = 4  # Relative permittivity of dielectric material
sigma = 10**(-5)  # Conductivity of dielectric material (S/m)
m = 1.67 * 10**(-27)  # Mass of air molecule (kg)
k = 1.38 * 10**(-23)  # Boltzmann constant (J/K)
T = 300  # Temperature of air (K)

Q_total = []
v_max = []

for E in E_rms:
    V = np.sqrt(2) * np.pi * 0.1 * E * L
    def electric_field(x, t):
        if 0 <= x <= d:
            return E * np.sin(2 * np.pi * f * t + 2 * np.pi * x / d)
        elif d < x <= L:
            return E * np.sin(2 * np.pi * f * t + 2 * np.pi * (L - x) / d)
        else:
            return 0

    def charge_density(x, t):
        return sigma * electric_field(x, t) / (2 * epsilon_0 * epsilon_r)

    def dbd_ode(y, x, t):
        V, Q = y
        dVdx = -charge_density(x, t) / epsilon_0
        dQdx = sigma * A * electric_field(x, t)
        return [dVdx, dQdx]

    y0 = [0, 0]
    x = np.linspace(0, L, 1000)
    t = np.linspace(0, 1 / f, 1000)

    sol = odeint(dbd_ode, y0, x, args=(t[0],))

    V = sol[:, 0]
    Q = sol[:, 1]

    # Calculate charge accumulation on dielectric surface
    Q_surf = np.sqrt(2 * epsilon_r * epsilon_0 * np.abs(V) * E * W)

    # Calculate gas speed
    mu = 1.458e-6 * T**(1.5) / (T + 110.4) * 10**(-3)
    v = np.sqrt(2 * Q_surf / (A * mu))
    v_max.append(np.max(v))

    # Add charge to total
    Q_total.append(np.trapz(Q_surf, x))

# Plot total charge deposited versus applied voltage
plt.figure()
plt.plot
